In [ ]:
from code_generate import Coder
from code_install_need_package import install_needed_package
from code_run import excute_python_code
from langchain_community.llms import Ollama
from langchain.agents import Tool

In [ ]:
llm = Ollama(model="qwen:7b")

### 定义Agent的Tools

In [3]:
coder = Coder()

code_generate_Tool = Tool(
    name = "请求Python代码助手",
    func = coder,
    description = "这是一个专业的Python代码助手，可以通过生成Python代码来解决你的代码问题，也能够分析代码错误信息，并进行代码修改。传入你的需求文本信息，或者代码执行错误信息。返回Python代码块。"
)

In [4]:
run_python_code_Tool = Tool(
    name = "Python代码执行工具",
    func = excute_python_code,
    description = "这是一个运行Python代码的工具，传入Python代码块，返回代码执行结果。"
)

In [5]:
install_package_Tool = Tool(
    name = "Python包安装工具",
    func = install_needed_package,
    description = "这是一个安装Python包的工具，当你缺少某个Python包时，帮你安装Python包，传入包名。"
)

In [6]:
Tools = (code_generate_Tool, run_python_code_Tool, install_package_Tool)

In [7]:
from langchain.agents import initialize_agent

In [8]:
code_agent = initialize_agent(
    tools=Tools,
    llm = llm,
    agent = "zero-shot-react-description",
    verbose= True,
    handle_parsing_errors=True
)

E:\My_Anaconda\Anaconda\envs\LangChain_and_VectorDB\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
from pprint import pprint

In [10]:
pprint(code_agent)

AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template="Answer the following questions as best you can. You have access to the following tools:\n\n请求Python代码助手(query: str) - 这是一个专业的Python代码助手，可以通过生成Python代码来解决你的代码问题，也能够分析代码错误信息，并进行代码修改。传入你的需求文本信息，或者代码执行错误信息。返回Python代码块。\nPython代码执行工具(code: str) - 这是一个运行Python代码的工具，传入Python代码块，返回代码执行结果。\nPython包安装工具(package_name: str, python_executable='E:\\\\My_Anaconda\\\\Anaconda\\\\envs\\\\LangChain_and_VectorDB\\\\python.exe') - 这是一个安装Python包的工具，当你缺少某个Python包时，帮你安装Python包，传入包名。\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [请求Python代码助手, Python代码执行工具, Python包安装工具]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can re

In [11]:
print(code_agent.agent.llm_chain.prompt.template) 

Answer the following questions as best you can. You have access to the following tools:

请求Python代码助手(query: str) - 这是一个专业的Python代码助手，可以通过生成Python代码来解决你的代码问题，也能够分析代码错误信息，并进行代码修改。传入你的需求文本信息，或者代码执行错误信息。返回Python代码块。
Python代码执行工具(code: str) - 这是一个运行Python代码的工具，传入Python代码块，返回代码执行结果。
Python包安装工具(package_name: str, python_executable='E:\\My_Anaconda\\Anaconda\\envs\\LangChain_and_VectorDB\\python.exe') - 这是一个安装Python包的工具，当你缺少某个Python包时，帮你安装Python包，传入包名。

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [请求Python代码助手, Python代码执行工具, Python包安装工具]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
# 修改template 让agent尽可能使用我们的工具
code_agent.agent.llm_chain.prompt.template = """请尽可能回答下列问题。您必须使用以下工具，你只能通过请求Python代码助手来生成代码:

请求Python代码助手(query: str) - 这是一个专业的Python代码助手，可以通过生成Python代码来解决你的代码问题，也能够分析代码错误信息，并进行代码修改。传入你的需求文本信息，或者代码执行错误信息。返回Python代码块。
Python代码执行工具(code: str) - 这是一个运行Python代码的工具，传入Python代码块，返回代码执行后的结果。
Python包安装工具(package_name: str, python_executable='E:\\My_Anaconda\\Anaconda\\envs\\LangChain_and_VectorDB\\python.exe') - 这是一个安装Python包的工具，当你缺少某个Python包时，帮你安装Python包，传入包名。

使用如下格式:

Question: 你必须回答的输入问题
Thought: 思考你要做什么
Action: 你采取的行动, 应该是这些中的其中一个 [请求Python代码助手, Python代码执行工具, Python包安装工具]
Action Input: 对action的输入
Observation: action的结果
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: 我现在知道最终的结果
Final Answer: 关于开始输入的问题的最终结果

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
code_agent.invoke("绘制一副回归图，并保存")